# Inference Server

This notebook start an HTTP server that poll in a directory "mypath" the presence of a image file ".jpg" and a ".ck" file with the same name of image file. Ex.: "DSC_005.jpg" and "DSC_005.jpg.ck".
If the files exist, it performs a classification of image and write a json file with this structure:  

Files after classification will be removed. 

Setup environment variable:  **SAMPLE_MAINTENANCE** for root project directory

In [ ]:
import os
import time
#testSet="close"
testSet="open"
sleepTime=0.05
mypath=os.environ["SAMPLE_MAINTENANCE"]+"server/static/img"
print (mypath)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
from os import listdir
from os.path import isfile, join
from PIL import Image
import classify
import pandas as pd
import matplotlib.pyplot as plt

while(True):
    time.sleep(sleepTime)
    for item in os.listdir(mypath):
        if (os.path.isdir(item)== False):
            if (item[-3:]=='.ck'):
                fileImg=mypath+'/'+item[:-3]
                if (os.path.isfile(fileImg)):
                    img = Image.open(fileImg)
                    width, height = img.size
                    ratio = height/width
                    width, height = img.size
                    imResized = img.resize((600,int(ratio*600)), Image.ANTIALIAS)
                    os.remove(mypath+"/"+item)
                    os.remove(fileImg)               
                    imResized.save(mypath+"/_"+item[:-3], 'JPEG', quality=90)
                    print("saved:"+mypath+"/_"+item[:-3])
                    predictions , label_lines = classify.Classify("/_"+item[:-3],mypath)
                    os.remove(mypath+"/_"+item[:-3])
                    print(predictions)
                    print(label_lines)
                    partJson= '{ \"name\":\"'+item[:-3]+'\",  \"results\": [ '
                    i=1
                    for r,l in zip(predictions[0],label_lines):
                        partJson+='{ \"class' +str(i) + '\":\"' + l + '\", \"value\":\"' + str(r) + '\" },' 
                        i+=1
                    json = partJson[:-1]+ ']  }'
                    print (json)
                    json_file = open(mypath+"/"+item[:-3]+".json", "w")
                    json_file.write(json)
                    json_file.close()